In [1]:
system("conda install -y r-sf=0.9_8")
system("conda install -y r-terra=1.2_10")

In [ ]:
# Title: Fig1 ----
# Arthur BARREAU: 06 Février 2025

# 1. LOAD LIBRARIES ----
library(terra)     
library(dplyr)
library(sf)        
library(jsonlite) 

# 2. LOAD DATA ----
# 2.1 DOWNLOAD AND LOAD ASD ----
output_dir <- "asd"
dir.create(output_dir, showWarnings = FALSE)

urls_asd <- c(
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG4326.shp",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG4326.shx",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG4326.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG4326.prj",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.shp",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.shx",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.prj"
)


destfiles_asd <- file.path(output_dir, basename(urls_asd))

for (i in seq_along(urls_asd)) {
  download.file(urls_asd[i], destfiles_asd[i], mode = "wb")
}

ASDs <- st_read(file.path(output_dir, "asd-shapefile-EPSG4326.shp"))

# Reproject ASD to the correct CRS (EASE-Grid 2.0 South)
ASDs <- st_transform(ASDs, crs = 6932)

# 2.2 DOWNLOAD AND LOAD CEMP ----
output_dir_cemp <- "cemp"
dir.create(output_dir_cemp, showWarnings = FALSE)

urls_cemp <- c(
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/CCAMLR_CEMP_EPSG4326.shp",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/CCAMLR_CEMP_EPSG4326.shx",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/CCAMLR_CEMP_EPSG4326.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/CCAMLR_CEMP_EPSG4326.prj"
)
destfiles_cemp <- file.path(output_dir_cemp, basename(urls_cemp))

for (i in seq_along(urls_cemp)) {
  download.file(urls_cemp[i], destfiles_cemp[i], mode = "wb")
}

CEMP <- st_read(file.path(output_dir_cemp, "cemp-shapefile-EPSG4326.shp"))

# Reproject CEMP to match ASDs' CRS
CEMP <- st_transform(CEMP, st_crs(ASDs))

# 2.3 DOWNLOAD AND LOAD BATHYMETRY ----
bathy_file <- "GEBCO2024_5000.tif"
options(timeout=180)
download.file("https://gis.ccamlr.org/geoserver/www/GEBCO2024_5000.tif", bathy_file, mode = "wb")
Bathy <- rast(bathy_file)


# 2.4 MASK 
json_path <- "galaxy_inputs/galaxy_inputs.json"
json_data <- fromJSON(json_path)
subarea <- json_data$subarea
area_list=resultat <- unlist(strsplit(subarea, "[,]"))

subareas_selected <- ASDs %>% filter(GAR_Short_ %in% area_list )
bbox <- st_bbox(subareas_selected)
bbox_polygon <- st_as_sfc(bbox)
bbox_vect <- vect(bbox_polygon)
Bathy_crop <- crop(Bathy, bbox_vect)  # Ajuste l'étendue
Bathy_mask <- mask(Bathy_crop, bbox_vect)  # Masque l'extérieur




Bathy_extent <- ext(Bathy_mask)
w= abs(Bathy_extent[2] - Bathy_extent[1])/10000
h=abs(Bathy_extent[4] - Bathy_extent[3])/10000 

png(filename = "outputs/test_Fig1.png", width = w, height = h)

# 4. MAP PLOTTING ----
Depth_cuts <- c(-8200 ,-7000 ,-6000 ,-5000, -4000, -3000, -1800, -1400, -1000,  -600,  -400 , -200  ,   0 ,   50  , 250   ,500)
Depth_cols <- c("#0000FF" ,"#061CFF", "#0C39FF", "#1256FF", "#1873FF" ,"#1E90FF", "#5CACEE", "#71BDF3", "#87CEFA", "#B0E2FF" ,"#D0EDFF", "#F0F8FF", "#FFFFFF", "#F2F2F2", "#E5E5E5")

# Plot bathymetry
plot(Bathy_mask,breaks = Depth_cuts, col = Depth_cols, legend = FALSE, axes = FALSE, box = FALSE,mar=c(0,0,0,0))

# Add ASD boundaries in red
plot(st_geometry(subareas_selected), add = TRUE, lwd = 0.75, border = 'red')

zones <- st_read("asd/asd-shapefile-EPSG6932.shp")    
centroids <- st_centroid(st_geometry(zones))
text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "red", cex = 1) 

# Add CEMP sites as orange points
plot(st_geometry(CEMP), add = TRUE, pch = 21, bg = 'orange', col = 'black', cex = 2.5)

# 5. SAVE OUTPUT ----

dev.off()





Warning message:
“package ‘terra’ was built under R version 4.0.5”
terra version 1.2.10


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0



Reading layer `asd-shapefile-EPSG4326' from data source `/data/jwd02f/pulsar_staging/79730487/working/jupyter/asd/asd-shapefile-EPSG4326.shp' using driver `ESRI Shapefile'
Simple feature collection with 19 features and 16 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -180 ymin: -85.41189 xmax: 180 ymax: -45
Geodetic CRS:  WGS 84
Reading layer `cemp-shapefile-EPSG4326' from data source `/data/jwd02f/pulsar_staging/79730487/working/jupyter/cemp/cemp-shapefile-EPSG4326.shp' using driver `ESRI Shapefile'
Simple feature collection with 32 features and 12 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -64.23 ymin: -77.2333 xmax: 170.2131 ymax: -46.7731
Geodetic CRS:  WGS 84
Simple feature collection with 10 features and 16 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: 1093794 ymin: -3329339 xmax: 4815055 ymax: 4234288
Projected CRS: WGS 84 / NSIDC EASE-Grid 2.0 South
   GAR_ID         GAR_Name GAR_Short_ GAR_Long_L GAR_Start_ 

Warning message in x@ptr$mask_vector(mask@ptr, inverse[1], updatevalue[1], touches[1], :
“GDAL Message 1: Failed to fetch spatial reference on layer lyr to build transformer, assuming matching coordinate systems.”
Warning message in plot.window(...):
“"box" is not a graphical parameter”
Warning message in plot.xy(xy, type, ...):
“"box" is not a graphical parameter”
Warning message in title(...):
“"box" is not a graphical parameter”


Reading layer `asd-shapefile-EPSG6932' from data source `/data/jwd02f/pulsar_staging/79730487/working/jupyter/asd/asd-shapefile-EPSG6932.shp' using driver `ESRI Shapefile'


Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
“GDAL Error 1: unhandled axis direction: "North along 90 deg East"”


Simple feature collection with 19 features and 16 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -3348556 ymin: -3329339 xmax: 4815055 ymax: 4371127
CRS:           NA


png 
  2